### data augumentation using Keras
on MNIST data


In [1]:
import numpy as np
#from tensorflow import keras
import keras

print('keras version:', keras.__version__)
print('keras backend:', keras.backend.backend())
print('keras image format:', keras.backend.image_data_format())

Using plaidml.keras.backend backend.


keras version: 2.2.4.1
keras backend: plaidml.keras.backend
keras image format: channels_last


In [2]:
data = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = data.load_data()

In [3]:
# show information about the downloaded data

print('train shape:', x_train.shape)
print('train y:', y_train.shape)
print('test_shape:', x_test.shape)
print('test y:', y_test.shape)


num_labels = len(np.unique(y_train))
image_size = x_train.shape[2]

print('num of labels:', num_labels)
print('image size:', image_size)

# calculate input shape and number of channels
is_channels_first = (keras.backend.image_data_format() == 'channels_first')
shape_len = len(x_train.shape)

if shape_len == 3:
    num_channels = 1
else:
    num_channels = 3

if is_channels_first:
    input_shape = (num_channels , image_szie , image_size)
else:
    input_shape = ( image_size , image_size , num_channels)

print('input shape:', input_shape)

train shape: (60000, 28, 28)
train y: (60000,)
test_shape: (10000, 28, 28)
test y: (10000,)
num of labels: 10
image size: 28
input shape: (28, 28, 1)


In [4]:
# convert the shape of data depending on the image data format

if is_channels_first :
    x_train2 = x_train.reshape(x_train.shape[0], num_channels, image_size, image_size)
    x_test2 = x_test.reshape(x_test.shape[0], num_channels, image_size, image_size)
else:
    x_train2 = x_train.reshape(x_train.shape[0], image_size, image_size, num_channels)
    x_test2 = x_test.reshape(x_test.shape[0], image_size, image_size, num_channels)


In [5]:
# normalize the data: 0.0 to 1.0

x_train2 = x_train2.astype('float32') / 255
x_test2 = x_test2.astype('float32') / 255

#hot encode
y_train2 = keras.utils.to_categorical(y_train)
y_test2 = keras.utils.to_categorical(y_test)

In [6]:
# show info about reshaped data

print("revised x_train shape:", x_train2.shape)
print('revised y_train shape:', y_train2.shape)
print('revised x_test shape:', x_test2.shape)
print('revised y_test shape:', y_test2.shape)
print('input shape:',input_shape)

revised x_train shape: (60000, 28, 28, 1)
revised y_train shape: (60000, 10)
revised x_test shape: (10000, 28, 28, 1)
revised y_test shape: (10000, 10)
input shape: (28, 28, 1)


In [7]:
num_hidden_layers = 512

print('hidden dense layer size:', num_hidden_layers)

hidden dense layer size: 512


In [8]:
#model
model = keras.models.Sequential()
model.add( keras.layers.Conv2D(32, kernel_size=(3,3), input_shape=input_shape , activation='relu' ))
model.add( keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add( keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu' ))
model.add( keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add( keras.layers.Dropout(0.5))

model.add( keras.layers.Flatten())
model.add( keras.layers.Dense(num_hidden_layers, activation='relu'))

model.add( keras.layers.Dropout(0.5))
model.add( keras.layers.Dense(10, activation='softmax'))

# compile to model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show summary
model.summary()

INFO:plaidml:Opening device "opencl_nvidia_quadro_p1000.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               819712    
__________

In [9]:
#train the model with train data for few runs to setup the weights

fit_history = model.fit(x_train2, y_train2,
epochs=10,
batch_size=128,
validation_data=(x_test2,y_test2)
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
59776/60000 [============================>.] - ETA: 0s - loss: 0.2547 - acc: 0.9210- ETA: 2s - loss: 0.2840 - acc: 0.911 

INFO:plaidml:Analyzing Ops: 131 of 249 operations complete


60000/60000 [==============================] - 18s 305us/step - loss: 0.2545 - acc: 0.9211 - val_loss: 0.0586 - val_acc: 0.9816
Epoch 2/10
60000/60000 [==============================] - 13s 221us/step - loss: 0.0879 - acc: 0.9729 - val_loss: 0.0362 - val_acc: 0.9865
Epoch 3/10
60000/60000 [==============================] - 14s 233us/step - loss: 0.0650 - acc: 0.9793 - val_loss: 0.0308 - val_acc: 0.9890
Epoch 4/10
60000/60000 [==============================] - 14s 234us/step - loss: 0.0548 - acc: 0.9827 - val_loss: 0.0269 - val_acc: 0.9897
Epoch 5/10
60000/60000 [==============================] - 15s 244us/step - loss: 0.0488 - acc: 0.9845 - val_loss: 0.0274 - val_acc: 0.9899
Epoch 6/10
60000/60000 [==============================] - 14s 228us/step - loss: 0.0435 - acc: 0.9856 - val_loss: 0.0220 - val_acc: 0.9911
Epoch 7/10
60000/60000 [==============================] - 13s 219us/step - loss: 0.0391 - acc: 0.9875 - val_loss: 0.0225 - val_acc: 0.9923
Epoch 8/10
60000/60000 [==============

In [10]:
# create dirs to hold logs and models so you can review them later

import os
import time

# helper function to create unique sub folder
def create_folder(folder_name):
    if (not os.path.exists(folder_name)):
        os.makedirs(folder_name)
    new_dir = folder_name + "/{}".format(time.time())
    if (not os.path.exists(new_dir)):
        os.makedirs(new_dir)
    return new_dir

log_dir = create_folder('logs')
model_dir = create_folder('models')


In [11]:
# define how to save snapshots of model states during training
checkpt_path=model_dir+'/va{val_acc:.5f}-ep{epoch:04d}-ac{acc:.5f}-vl{val_loss:.5f}-l{loss:.5f}.hdf5'
cp_callback = keras.callbacks.ModelCheckpoint(
  checkpt_path ,
  verbose=1
)

In [12]:
# setup to capture log so that you can view it from tensorflow dashboard
# comment this out if you are NOT using tensorflow

#import tensorflow as tf
#tf_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)


In [13]:
# use datagen to 'augument' the training data
# tweak the parameters to get better results

datagen = keras.preprocessing.image.ImageDataGenerator(
  rotation_range = 25,
  width_shift_range=0.10,
  height_shift_range=0.10,
  zoom_range = 0.10
)

In [14]:
batch_size = 128
epochs = 300

print('batch size:', batch_size)
print('epochs:', epochs)

batch size: 128
epochs: 300


In [15]:
# run data augumentation based training
# add in tf_callback if using tensorflow so you can monitor the progress using tf dashboard

fit_history2 = model.fit_generator(
  datagen.flow(x_train2,y_train2,batch_size=batch_size),
  steps_per_epoch = int(len(x_train2) / batch_size),
  epochs = epochs,
  validation_data = (x_test2, y_test2),
  #workers=4, 
  callbacks=[cp_callback]
)

Epoch 1/300
468/468 [==============================] - 27s 58ms/step - loss: 0.2161 - acc: 0.9344 - val_loss: 0.0247 - val_acc: 0.9929

Epoch 00001: saving model to models/1557955871.1748924/va0.99290-ep0001-ac0.93441-vl0.02472-l0.21615.hdf5
Epoch 2/300
468/468 [==============================] - 25s 54ms/step - loss: 0.1384 - acc: 0.9574 - val_loss: 0.0209 - val_acc: 0.9935

Epoch 00002: saving model to models/1557955871.1748924/va0.99350-ep0002-ac0.95738-vl0.02088-l0.13839.hdf5
Epoch 3/300
468/468 [==============================] - 29s 63ms/step - loss: 0.1173 - acc: 0.9640 - val_loss: 0.0216 - val_acc: 0.9933

Epoch 00003: saving model to models/1557955871.1748924/va0.99330-ep0003-ac0.96401-vl0.02159-l0.11729.hdf5
Epoch 4/300
468/468 [==============================] - 26s 55ms/step - loss: 0.1049 - acc: 0.9686 - val_loss: 0.0213 - val_acc: 0.9926

Epoch 00004: saving model to models/1557955871.1748924/va0.99260-ep0004-ac0.96858-vl0.02130-l0.10483.hdf5
Epoch 5/300
 57/468 [==>........

KeyboardInterrupt: 